In [2]:
import pandas as pd
import numpy as np
import math
from datetime import datetime

In [3]:
super_dataset = pd.read_csv('../data/superDataset.csv')
world_dataset = pd.read_csv('../data/owid-covid-data.csv')

In [7]:
countries = ['Indonesia', 'Pakistan', 'Nigeria', 'Brazil', 'Bangladesh', 'United States']

filtered_dataset = world_dataset[world_dataset.location.isin(countries)]
filtered_dataset.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,new_vaccinations_smoothed_per_million,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
5689,BGD,Asia,Bangladesh,2020-03-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108.0,108.0,0.001,0.001,NaN,NaN,NaN,NaN,tests performed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.89,164689383.0,1265.036,27.5,5.098,3.262,3523.984,14.8,298.003,8.38,1.0,44.7,34.808,0.8,72.59,0.632
5690,BGD,Asia,Bangladesh,2020-03-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,111.0,0.001,0.000,NaN,NaN,NaN,NaN,tests performed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.89,164689383.0,1265.036,27.5,5.098,3.262,3523.984,14.8,298.003,8.38,1.0,44.7,34.808,0.8,72.59,0.632
5691,BGD,Asia,Bangladesh,2020-03-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tests performed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.89,164689383.0,1265.036,27.5,5.098,3.262,3523.984,14.8,298.003,8.38,1.0,44.7,34.808,0.8,72.59,0.632
5692,BGD,Asia,Bangladesh,2020-03-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,120.0,0.001,0.000,NaN,NaN,NaN,NaN,tests performed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.89,164689383.0,1265.036,27.5,5.098,3.262,3523.984,14.8,298.003,8.38,1.0,44.7,34.808,0.8,72.59,0.632
5693,BGD,Asia,Bangladesh,2020-03-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,127.0,0.001,0.000,NaN,NaN,NaN,NaN,tests performed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.89,164689383.0,1265.036,27.5,5.098,3.262,3523.984,14.8,298.003,8.38,1.0,44.7,34.808,0.8,72.59,0.632


In [ ]:
us_dataset = filtered_dataset[filtered_dataset.location == 'United States'].fillna(0)
us_dataset.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,new_vaccinations_smoothed_per_million,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
68974,USA,North America,United States,2020-01-22,1.0,0.0,0.0,0.0,0.0,0.0,0.003,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,331002647.0,35.608,38.3,15.413,9.732,54225.446,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926
68975,USA,North America,United States,2020-01-23,1.0,0.0,0.0,0.0,0.0,0.0,0.003,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,331002647.0,35.608,38.3,15.413,9.732,54225.446,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926
68976,USA,North America,United States,2020-01-24,2.0,1.0,0.0,0.0,0.0,0.0,0.006,0.003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,331002647.0,35.608,38.3,15.413,9.732,54225.446,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926
68977,USA,North America,United States,2020-01-25,2.0,0.0,0.0,0.0,0.0,0.0,0.006,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,331002647.0,35.608,38.3,15.413,9.732,54225.446,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926
68978,USA,North America,United States,2020-01-26,5.0,3.0,0.0,0.0,0.0,0.0,0.015,0.009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,331002647.0,35.608,38.3,15.413,9.732,54225.446,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926


In [ ]:
us_dataset[['year', 'week', 'day']] = pd.DataFrame(us_dataset.date.apply(lambda x: datetime.strptime(x, "%Y-%m-%d").isocalendar()).to_list(), index=us_dataset.index)
us_dataset.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,new_vaccinations_smoothed_per_million,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,year,week,day
68974,USA,North America,United States,2020-01-22,1.0,0.0,0.0,0.0,0.0,0.0,0.003,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,331002647.0,35.608,38.3,15.413,9.732,54225.446,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,2020,4,3
68975,USA,North America,United States,2020-01-23,1.0,0.0,0.0,0.0,0.0,0.0,0.003,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,331002647.0,35.608,38.3,15.413,9.732,54225.446,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,2020,4,4
68976,USA,North America,United States,2020-01-24,2.0,1.0,0.0,0.0,0.0,0.0,0.006,0.003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,331002647.0,35.608,38.3,15.413,9.732,54225.446,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,2020,4,5
68977,USA,North America,United States,2020-01-25,2.0,0.0,0.0,0.0,0.0,0.0,0.006,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,331002647.0,35.608,38.3,15.413,9.732,54225.446,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,2020,4,6
68978,USA,North America,United States,2020-01-26,5.0,3.0,0.0,0.0,0.0,0.0,0.015,0.009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,331002647.0,35.608,38.3,15.413,9.732,54225.446,1.2,151.089,10.79,19.1,24.6,0.0,2.77,78.86,0.926,2020,4,7


In [ ]:
# us_dataset['date'] = pd.to_datetime(us_dataset.date)
# us_dataset['week'] = us_dataset.date.dt.isocalendar().week

In [ ]:
weekly_data = pd.DataFrame(us_dataset.groupby(['year', 'week']).mean().new_cases.apply(lambda x: math.ceil(x))).reset_index()
weekly_data.head()

,year,week,new_cases
0,2020,4,1
1,2020,5,1
2,2020,6,1
3,2020,7,1
4,2020,8,1


In [31]:
main_table = filtered_dataset.fillna(0)

In [32]:
all_cases = main_table[['location','date','new_cases','population']]

In [33]:
all_cases

,location,date,new_cases,population
5689,Bangladesh,2020-03-03,0.0,164689383.0
5690,Bangladesh,2020-03-04,0.0,164689383.0
5691,Bangladesh,2020-03-05,0.0,164689383.0
5692,Bangladesh,2020-03-06,0.0,164689383.0
5693,Bangladesh,2020-03-07,0.0,164689383.0
...,...,...,...,...
69377,United States,2021-02-28,51204.0,331002647.0
69378,United States,2021-03-01,58765.0,331002647.0
69379,United States,2021-03-02,56758.0,331002647.0
69380,United States,2021-03-03,65909.0,331002647.0


In [79]:
all_cases['norm']=(all_cases.new_cases/all_cases.population)*1000000

In [84]:
all_cases['log'] = np.log(all_cases['norm']).replace(-np.inf,0)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [87]:
all_cases.head(20)

,location,date,new_cases,population,norm,log
5689,Bangladesh,2020-03-03,0.0,164689383.0,0.000000,0.000000
5690,Bangladesh,2020-03-04,0.0,164689383.0,0.000000,0.000000
5691,Bangladesh,2020-03-05,0.0,164689383.0,0.000000,0.000000
5692,Bangladesh,2020-03-06,0.0,164689383.0,0.000000,0.000000
5693,Bangladesh,2020-03-07,0.0,164689383.0,0.000000,0.000000
5694,Bangladesh,2020-03-08,3.0,164689383.0,0.018216,-4.005449
5695,Bangladesh,2020-03-09,0.0,164689383.0,0.000000,0.000000
5696,Bangladesh,2020-03-10,0.0,164689383.0,0.000000,0.000000
5697,Bangladesh,2020-03-11,0.0,164689383.0,0.000000,0.000000
5698,Bangladesh,2020-03-12,0.0,164689383.0,0.000000,0.000000


In [21]:
all_deaths = main_table[['location','date','new_deaths','population']] 

In [22]:
all_deaths

,location,date,new_deaths,population
5689,Bangladesh,2020-03-03,0.0,164689383.0
5690,Bangladesh,2020-03-04,0.0,164689383.0
5691,Bangladesh,2020-03-05,0.0,164689383.0
5692,Bangladesh,2020-03-06,0.0,164689383.0
5693,Bangladesh,2020-03-07,0.0,164689383.0
...,...,...,...,...
69377,United States,2021-02-28,1037.0,331002647.0
69378,United States,2021-03-01,1516.0,331002647.0
69379,United States,2021-03-02,1924.0,331002647.0
69380,United States,2021-03-03,2468.0,331002647.0


In [65]:
all_deaths['norm']=(all_deaths.new_deaths/all_deaths.population)*1000000

In [66]:
all_deaths['log'] = np.log(all_deaths['norm']).replace(-np.inf,0)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [89]:
all_deaths

,location,date,new_deaths,population,norm,log
5689,Bangladesh,2020-03-03,0.0,164689383.0,0.000000,0.000000
5690,Bangladesh,2020-03-04,0.0,164689383.0,0.000000,0.000000
5691,Bangladesh,2020-03-05,0.0,164689383.0,0.000000,0.000000
5692,Bangladesh,2020-03-06,0.0,164689383.0,0.000000,0.000000
5693,Bangladesh,2020-03-07,0.0,164689383.0,0.000000,0.000000
...,...,...,...,...,...,...
69377,United States,2021-02-28,1037.0,331002647.0,3.132905,1.141961
69378,United States,2021-03-01,1516.0,331002647.0,4.580024,1.521704
69379,United States,2021-03-02,1924.0,331002647.0,5.812642,1.760035
69380,United States,2021-03-03,2468.0,331002647.0,7.456134,2.009037
